# Testing LeNet-5
A tutorial to test LeNet-5 using ADAPT. Before start tutorial, if you use your GPU, the following cell will set tensorflow to use minimal memory.

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

## Load model to test
Here, we use LeNet-5 and we offers how to create a LeNet-5 model.

### 1. Load MNIST
You can easily load MNIST data supplyied by Tensorflow/Keras.

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

Normalize the values of each pixel into 0 and 1, and convert the label to categorical vector.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 2. Create model
The structure of the model is originally from the following paper:

    Gradient-based learning applied to document recognition

In [ ]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
def LeNet5():
    
    # Input layer.
    input_tensor = Input(shape=(28, 28, 1))
    
    # Block 1.
    x = Convolution2D(6, (5, 5), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2), name='block1_pool1')(x)
    
    # Block 2.
    x = Convolution2D(16, (5, 5), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='block2_pool1')(x)
    
    # Fully connected.
    x = Flatten(name='flatten')(x)
    x = Dense(120, activation='relu', name='fc1')(x)
    x = Dense(84, activation='relu', name='fc2')(x)
    x = Dense(10, name='before_softmax')(x)
    x = Activation('softmax', name='redictions')(x)
    
    return Model(input_tensor, x)

In [ ]:
model = LeNet5()
model.summary()

### 3. Train model
Train the model with the adam optimizer for 10 epochs.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_ = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10, verbose=1)

The final accuracy of the trained model is about 99%.

## Test using ADAPT
From now on, let's test the created model.

### 1. Choose candidate input
In this tutorial, the 9721<sup>st</sup> image from the test set will be used.
Here is how it looks.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
idx = 9721
image = x_test[idx]
plt.axis('off')
plt.imshow(image.reshape(28, 28), cmap='gray')
print('Label:', np.argmax(y_test[idx]))

The chosen inputs label is 7 and it looks like 7.

### 2. Create a fuzzer
ADAPT offers various modules (e.g. coverage metrics and neuron selection strategies) that can be used to compose a fuzzer. First thing to do is wrapping Keras model, since all modules in ADAPT uses a Keras model wrapped with the ```adapt.Network``` class.

In [ ]:
from adapt import Network

In [ ]:
network = Network(model)

Create a neuron coverage with 0.5 as threshold.

In [ ]:
from adapt.metric import NC

In [ ]:
metric = NC(0.5)

Create an adaptive and parameterized neuron selection strategy introduced in the following paper:

    Effective White-box Testing of Deep Neural Networks with Adaptive Neuron-Selection Strategy

In [ ]:
from adapt.strategy import AdaptiveParameterizedStrategy

In [ ]:
strategy = AdaptiveParameterizedStrategy(network)

Now you can compose a fuzzer for the LeNet-5 with the 3592<sup>nd</sup> test input.

In [ ]:
from adapt.fuzzer import WhiteBoxFuzzer

In [ ]:
fuzzer = WhiteBoxFuzzer(network, image, metric, strategy)

### 3. Start testing
The given input is tested for 10 minutes, and keep all the inputs generated.

In [ ]:
archive = fuzzer.start(minutes=10, append='all')

### 4. Testing result
You can easily see the summary of the testing result as follows:

In [ ]:
archive.summary()

Here is the coverage graph.

In [ ]:
t, cov = tuple(zip(*archive.timestamp))
plt.plot(t, cov)

Plus, the following is a visualization of some generated images.

In [ ]:
fig, ax = plt.subplots(1, len(archive.found_labels), figsize=(len(archive.found_labels) * 2, 2))
for i, label in enumerate(archive.found_labels.keys()):
    ax[i].set_axis_off()
    ax[i].title.set_text(str(label))
    im = archive.inputs[label][0]
    im = np.reshape(im, (28, 28)).clip(0, 1)
    ax[i].imshow(np.reshape(archive.inputs[label][0], (28, 28)), cmap='gray')